(***) Un equipo del Ministerio de Salud de la Nación se encuentra realizando distintos análisis de datos a partir de la información que tienen unificada a nivel nacional de los sistemas de información para el seguimiento de la campaña de vacunación contra el sars-cov-2.

Es por ello que cuentan con información en los siguientes archivos csv:

```
centro_de_vacunacion.csv (id_centro_vacunacion, nombre, direccion, codigo_postal, partido, provincia)
```

```
categoria_plan_vacunacion.csv  (id_categoria, nombre_categoria, prioridad)
```

```
aplicaciones.csv (id_ciudadano, fecha_aplicacion, id_categoria, id_centro_vacunacion, id_unico_dosis_vacuna, id_tipo_vacuna)
```
<blockquote>
El primer archivo cuenta con información de los distintos centros de vacunación a lo largo y ancho del país donde se realizan aplicaciones de las vacunas disponibles. Este archivo nos da información geográfica de donde se encuentran los centros de vacunación en cada provincia.

El segundo tiene información de las distintas categorías que tiene el plan de vacunación, y la prioridad de las mismas, las cuales son valores numéricos siendo el valor 1 el valor más alto de prioridad. Por ejemplo, la categoría “Personal Esencial: Salud” tiene prioridad 1. Tener en cuenta que múltiples categorías pueden tener el mismo número de prioridad.

En el último archivo, contamos con la información de ciudadanos que se fueron realizando aplicaciones dentro del plan de vacunación, cuando se aplicó la misma (en formato “YYYY-mm-dd”), en que lugar y bajo qué categoría se hizo. Algo a considerar aquí es que también contamos con información específica de la dosis que se le aplicó a la personas y el id del tipo de vacuna.

Algo importante a considerar es que en la Argentina solamente se cuenta por el momento con vacunas que requieren 2 dosis para tener inmunidad completa, es por eso que en aplicaciones.csv pueden haber múltiples entradas por ciudadano.

Se desea:

a) Indicar el porcentaje de ciudadanos vacunados en prioridad 1 que tienen inmunidad completa (recibieron dos aplicaciones de vacuna).

b) Indicar cual es el top 5 de provincias que han inmunizado de forma completa a mayor cantidad de ciudadanos (es decir realizando dos aplicaciones de la vacuna) en el periodo temporal de diciembre de 2020 a marzo (inclusive) de 2021.
</blockquote>


**<blockquote>
    a) Indicar el porcentaje de ciudadanos vacunados en prioridad 1 que tienen inmunidad completa (recibieron dos aplicaciones de vacuna).
    </blockquote>**

In [173]:
import pandas as pd 
import numpy as np
import random as rd

In [174]:
id_ciudadano = np.random.randint(1, high = 10, size = 20)
id_centro_vacunacion = rd.sample(range(1,21),20)
id_categoria = rd.sample(range(1,21),20)
id_provincia = np.random.randint(1, high = 5, size = 20)


centros_data = {
        'id_centro_vacunacion': id_centro_vacunacion,
        'nombre': id_centro_vacunacion,
        'direccion': id_centro_vacunacion,
        'codigo_postal': id_centro_vacunacion,
        'partido': id_centro_vacunacion,
        'provincia': id_provincia, 
}

categoria_data = {
        'id_categoria': id_categoria,
        'nombre_categoria': id_categoria,
        'prioridad': np.random.randint(1, high=5, size=20),
}

aplicaciones_data = {
        'id_ciudadano': id_ciudadano,
        'fecha': pd.date_range(start = '2020-11-01', end='2021-04-01', periods=20),
        'id_categoria': id_categoria,
        'id_centro_vacunacion': id_centro_vacunacion,
        'id_unico_dosis_vacuna': rd.sample(range(1,199), 20),
        'id_tipo_vacuna': np.random.randint(1,high=3, size=20), 
}

In [175]:
centro_vacunacion = pd.DataFrame(centros_data, columns=centros_data.keys())
categoria_plan_vacunacion = pd.DataFrame(categoria_data, columns=categoria_data.keys())
aplicaciones = pd.DataFrame(aplicaciones_data, columns=aplicaciones_data.keys())

In [176]:
#quiero la cantidad de ciudadanos para luego hacer el porcentaje de full inmunizados, y elimino duplicados para no contar de mas 
cantidadTotalCiudadanos = len(aplicaciones)
cantidadTotalCiudadanos

20

In [177]:
#primero filtro ciudadanos inmunizados
aplicaciones['cantidad_dosis_aplicadas'] = aplicaciones.groupby('id_ciudadano')['id_unico_dosis_vacuna'].transform('count')
ciudadanos_inmunizados = aplicaciones.loc[aplicaciones['cantidad_dosis_aplicadas'] == 2, ['id_ciudadano','id_categoria','id_centro_vacunacion']]

In [178]:
#segundo filtro por personas vacunadas con priorida uno 
ciudadanos_prioritarios = categoria_plan_vacunacion.loc[categoria_plan_vacunacion['prioridad'] == 1, ['id_categoria','nombre_categoria']]

In [179]:
#hago un merge entre estos dateframes, en este caso usando un inner
resultado_A = ciudadanos_prioritarios.merge(ciudadanos_inmunizados, how = 'inner')
cantidadCiudadanosInmunizados = resultado_A['id_ciudadano'].count()

porcentajeInmunizados = (cantidadCiudadanosInmunizados*100)/cantidadTotalCiudadanos
print('El porcentaje de ciudadanos inmunizados con prioridad 1 es %',porcentajeInmunizados)

El porcentaje de ciudadanos inmunizados con prioridad 1 es % 20.0


**<blockquote>
    b) Indicar cual es el top 5 de provincias que han inmunizado de forma completa a mayor cantidad de ciudadanos (es decir realizando dos aplicaciones de la vacuna) en el periodo temporal de diciembre de 2020 a marzo (inclusive) de 2021.
    </blockquote>**

In [180]:
#convierto al tipo datetime la columna fecha_aplicacion del df aplicaciones
aplicaciones['fecha'] = pd.to_datetime(aplicaciones['fecha'])
fecha_comienzo = pd.to_datetime('2020-12-01')
fecha_final = pd.to_datetime('2021-03-31')
periodo_temporal = (fecha_comienzo <= aplicaciones['fecha']) & (aplicaciones['fecha'] <= fecha_final)
aplicaciones_en_periodo_deseado = aplicaciones.loc[periodo_temporal, ['id_centro_vacunacion','id_categoria']]
aplicaciones_en_periodo_deseado

,id_centro_vacunacion,id_categoria
4,16,11
5,7,5
6,17,14
7,12,4
8,5,1
9,10,15
10,14,2
11,15,17
12,11,3
13,20,7


In [181]:
#reutilizo el df de ciudadanos_inmunizados para mergear

resultado_B = ciudadanos_inmunizados.merge(aplicaciones_en_periodo_deseado, how = 'inner')
resultado_B

,id_ciudadano,id_categoria,id_centro_vacunacion
0,8,11,16
1,2,14,17
2,4,1,5
3,2,7,20
4,3,20,19
5,1,18,13
6,8,8,2
7,1,6,4


In [182]:
#ahora hago un merge con la clave 'id_centro_vacunacion'
resultado_B = centro_vacunacion.merge(resultado_B, how = 'inner')
#en este caso el join puedo hacer tanto inner como right
resultado_B = resultado_B.loc[:,['id_centro_vacunacion','provincia']]

In [183]:
#agrego una columna que me cuente la cantidad de vacunados por provincia
resultado_B['cantidad_vacunados'] = resultado_B.groupby('provincia').transform('count')
resultado_B.drop_duplicates('provincia').nlargest(5,'cantidad_vacunados').loc[:,['provincia','cantidad_vacunados']]

,provincia,cantidad_vacunados
0,2,3
3,4,3
2,3,2
